In [3]:
import csv
from io import BytesIO
import requests
import pdfplumber
import os
import pikepdf
import pandas as pd
import tabula
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader, PdfWriter

In [100]:
# Abrir o arquivo HTML bruto
with open('htmlBruto.txt', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Usar BeautifulSoup para analisar o HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Encontrar todas as divs com a classe 'views-row'
views_rows = soup.find_all('div', class_='views-row')
i = 0
# Abrir o arquivo de texto para escrita
with open('links_eventosExcecionais.txt', 'w', encoding='utf-8') as output_file:
    # Iterar sobre as divs 'views-row' encontradas
    for row in views_rows:
        # Encontrar todos os links dentro da div 'views-row'
        links = row.find_all('a', href=True)
        # Extrair os URLs dos links
        for link in links:
            href = link['href']
            formatted_link = f"https://www.e-redes.pt{href}"
            print(formatted_link)
            output_file.write(formatted_link + '\n')
            i+=1

print(f"{i} links foram extraídos e gravados no arquivo 'links_extraidos.txt'.")


https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Depress%C3%A3o%20Ciar%C3%A1n%202023.pdf
https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Depress%C3%A3o%20Babet%202023.pdf
https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Intemp%C3%A9rie%20In%C3%ADcio%20do%20Ano%202023.pdf
https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Junho%202023.pdf
https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Maio%202023.pdf
https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Abril%202023.pdf
https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Mar%C3%A7o%202023.pdf
https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Fevereiro%202023.pdf
https://www.e

- - -

In [101]:
with open('links_eventosExcecionais.txt', 'r') as txt:
    links = [line.strip() for line in txt]

# Diretório para salvar os PDFs baixados
output_directory = "pdfs"

# Certificar-se de que o diretório de saída existe
os.makedirs(output_directory, exist_ok=True)

# Lista para armazenar os links que não contêm tabelas
links_sem_tabela = []

# Lista para armazenar os DataFrames dos PDFs que contêm tabelas
dfs = []

# Colunas do DataFrame
nomes_colunas = ['Código do Relatório', 'Concelho origem do incidente', 'Data do incidente',
                  'Nível de Tensão', 'Causa do incidente', 'Duração incidente (min)',
                  'N.º clientes afectados', 'TIEPI MT(min)', 'SAIFI MT(#)', 'SAIDI MT(min)',
                  'END(MWh)', 'SAIFI BT(#)', 'SAIDI BT(min)', 'Qualidade de Energia Elétrica',
                  'Qualidade de Serviço Comercial', 'Fundamentação', 'Decisão preliminar']

# Baixar e extrair tabelas para cada link
for link in links:
    # Extrair o nome do arquivo do link
    nome_arquivo = link.split('/')[-1]

    # Caminho do arquivo para salvar
    caminho_arquivo = os.path.join(output_directory, nome_arquivo)

    # Baixar o PDF
    with open(caminho_arquivo, 'wb') as arquivo:
        resposta = requests.get(link)
        arquivo.write(resposta.content)

    # Extrair tabelas do PDF baixado
    try:
        # Extrair tabelas de todas as páginas do PDF
        dfs_paginas = tabula.read_pdf(caminho_arquivo, pages='all', multiple_tables=True)
        if len(dfs_paginas) > 0:
            # Iterar sobre os DataFrames de cada página
            for df_pagina in dfs_paginas:
                if 'Código do\rIncidente' in df_pagina.columns or 'Incidente' in df_pagina.columns:
                    # Remover a coluna antiga se ela existir
                    if 'Código do\rIncidente' in df_pagina.columns:
                        df_pagina.drop(columns='Código do\rIncidente', inplace=True)
                    if 'Incidente' in df_pagina.columns:
                        df_pagina.drop(columns='Incidente', inplace=True)

                if len(df_pagina.columns) < len(nomes_colunas):
                    df_pagina.insert(13, 'Nova Coluna', "n.a.")
                
                # Renomear as colunas
                df_pagina.columns = nomes_colunas
                
                # Remover as três primeiras linhas
                df_pagina = df_pagina.iloc[3:].copy()

                # Adicionar uma nova coluna 'Link' com o link correspondente
                df_pagina['Link'] = link
                
                # Adicionar o DataFrame à lista
                dfs.append(df_pagina)
        else:
            print(f"PDF sem tabelas: {link}. Tabelas: {len(dfs_paginas)}")
            links_sem_tabela.append(link)
    except Exception as e:
        print(f"Erro ao processar o PDF {link}: {e}")
        links_sem_tabela.append(link)

# Concatenar todos os DataFrames em um único DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Exibir DataFrame
print("")
print(df_final)

# Exibir links que não contêm tabelas
print("\nLinks sem tabelas:")
for link in links_sem_tabela:
    print(link)


Erro ao processar o PDF https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Depress%C3%A3o%20Ciar%C3%A1n%202023.pdf: index 13 is out of bounds for axis 0 with size 4
PDF sem tabelas: https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Depress%C3%A3o%20Babet%202023.pdf. Tabelas: 0
PDF sem tabelas: https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Intemp%C3%A9rie%20In%C3%ADcio%20do%20Ano%202023.pdf. Tabelas: 0
PDF sem tabelas: https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Junho%202023.pdf. Tabelas: 0
PDF sem tabelas: https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Maio%202023.pdf. Tabelas: 0
PDF sem tabelas: https://www.e-redes.pt/sites/eredes/files/2024-03/Evento%20Excecional%20Tabela%20Decis%C3%A3o%20Abril%202023.pdf. Tabelas: 0
Erro ao processar o PDF https

In [102]:
df_final

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afectados,TIEPI MT(min),SAIFI MT(#),SAIDI MT(min),END(MWh),SAIFI BT(#),SAIDI BT(min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão preliminar,Link
0,EREDES_2023_NOV_TC_1,Barcelos,1/11/2023 22:30,AT - Alta tensão,VENTO INTENSIDADE EXCECIONAL,870.0,244 990,"1,807","0,053","2,822","141,327","0,049","2,667","Exclusão, para efeitos de verificação do cumpr...",Exclusão do pagamento de eventuais compensaçõe...,Reúne condições para aprovação:\r- Reúne os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2024...
1,EREDES_2022_DEZ_TC_2,Território de\rPortugal continental,11/12/2022\r00:00,AT - Alta tensão,VENTO INTENSIDADE EXCECIONAL,2280.0,387223,"4,700","0,080","6,840","390,700","0,070","5,810","Exclusão, para efeitos de verificação do cumpr...",Exclusão do pagamento de eventuais compensaçõe...,Reúne condições para aprovação:\r- Reúne os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2023...
2,EREDES_2021_OUT_TC_1,Território de\rPortugal continental,29/10/2021\r19:30,AT - Alta tensão,VENTO INTENSIDADE EXCECIONAL,1320.0,237413,"3,081","0,079","4,384","200,406","0,054","2,909","Exclusão, para efeitos de verificação do cumpr...",Exclusão do pagamento de eventuais compensaçõe...,Reúne condições para aprovação:\r- Reúne os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2022...
3,EREDES_2021_JUL_TC_1,Portugal continental,24/071/2021\r15:36,AT - Alta tensão,DESLASTRE DE CARGA AUTOMÁTICO,126.0,977394,"7,454","0,186","8,132","506,029","0,156","6,894",n.a.,Atendimento telefónico para comunicação\rde av...,Reúne condições para aprovação:\r- Reúne os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2022...
4,EDPD_2021_JAN_TC_1,Território de\rPortugal continental,21/01/2021\r22:30,AT - Alta tensão,VENTO INTENSIDADE EXCECIONAL,690.0,269663,"1,730","0,060","3,680","126,030","0,050","2,540","Exclusão, para efeitos de verificação do cumpr...",Exclusão do pagamento de eventuais compensaçõe...,Reúne condições para aprovação:\r- Reúne os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2022...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970,EDPD_2014_JUN_T_114,VISEU,2014-06-24,MT,ABATE DE ÁRVORES,173.0,3652,"0,014600","0,000218","0,025300","0,95","0,000259","0,023300",n.a.,n.a.,Reune condições para aprovação:\r- Reune os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2019...
2971,EDPD_2014_JUN_T_118,FIGUEIRA DA FOZ,2014-06-07,MT,ABATE DE ÁRVORES,168.0,2055,"0,006640","0,000276","0,021000","0,43","0,000235","0,017800",n.a.,n.a.,Reune condições para aprovação:\r- Reune os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2019...
2972,EDPD_2014_JUN_T_124,PORTO DE MÓS,2014-06-18,MT,DESLIZAMENTO DE TERRAS,274.0,1834,"0,010100","0,000345","0,027800","0,65","0,000302","0,020900",n.a.,n.a.,Reune condições para aprovação:\r- Reune os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2019...
2973,EDPD_2014_JUN_T_126,VISEU,2014-06-26,MT,ABATE DE ÁRVORES,159.0,4781,0,"0,000207","0,011600",0,"0,000205","0,011100",n.a.,n.a.,Reune condições para aprovação:\r- Reune os re...,Aprovado,https://www.e-redes.pt/sites/eredes/files/2019...


In [103]:
df_final.to_excel('Scrapping.xlsx', index=False)

In [104]:
df_final.to_csv('Scrapping.csv', index=False)